# IBM Circuit function

## Overview

The IBM&reg; Circuit function takes [abstract PUBs](./primitive-input-output) as inputs, and returns mitigated expectation values as outputs. This circuit function includes an automated and customized pipeline to enable researchers to focus on algorithm and application discovery.

## Description

After submitting your PUB, your abstract circuits and observables are automatically transpiled, executed on hardware, and post-processed to return mitigated expectation values. To do so, this combines the following tools:

- [Qiskit Transpiler Service](./qiskit-transpiler-service), including auto-selection of AI-driven and heuristic transpilation passes to translate your abstract circuits to [hardware-optimized ISA circuits](/guides/optimize-for-hardware)
- [Error suppression and mitigation required for utility-scale computation](./error-mitigation-and-suppression-techniques), including measurement and gate twirling, dynamical decoupling, Twirled Readout Error eXtinction (TREX), Zero-Noise Extrapolation (ZNE), and Probabilistic Error Amplification (PEA)
- [Qiskit Runtime Estimator](./get-started-with-primitives), to execute ISA PUBs on hardware and return mitigated expectation values

![IBM Circuit function](/images/guides/ibm-circuit-function/ibm-circuit-function.svg)

## Get started

Authenticate using your [IBM Quantum&trade; Platform API token](http://quantum.ibm.com/), and select the Qiskit Function as follows:

In [ ]:
from qiskit_ibm_catalog import QiskitFunctionCatalog

catalog = QiskitFunctionCatalog()
ibm_cf = catalog.load("ibm/circuit-function")

## Example

 To get started, try this basic example:

In [ ]:
from qiskit.circuit.random import random_circuit
from qiskit_ibm_runtime import QiskitRuntimeService

# You can skip this step if you have a target backend, e.g.
# backend_name = "ibm_sherbrooke"
# You'll need to specify the credentials when initializing QiskitRuntimeService, if they were not previously saved.
service = QiskitRuntimeService()
backend = service.least_busy(operational=True, simulator=False)

circuit = random_circuit(num_qubits=2, depth=2, seed=42)
observable = "Z" * circuit.num_qubits
pubs = [(circuit, observable)]

job = function.run(
  backend=backend.name,  # Or `backend=backend_name`, if you didn't initialize a backend object
  pubs=pubs
)

Check your Qiskit Function workload's [status](/guides/functions#check-job-status) or return [results](/guides/functions#retrieve-results) as follows:

In [ ]:
print(job.status())
result = job.result()

The results have the same format as an [Estimator result](https://docs.quantum.ibm.com/guides/primitive-input-output#estimator-output):

In [ ]:
print(f'The result of the submitted job had {len(result)} PUB and has a value:\n {result}\n')
print(f'The associated PubResult of this job has the following DataBins:\n {result[0].data}\n')
print(f'And this DataBin has attributes: {result[0].data.keys()}')
print(f'The expectation values measured from this PUB are: \n{result[0].data.evs}')

## Parameters

See the following table for all input parameters the IBM Circuit function accepts. The subsequent [Options](#options) section goes into more details about the available `options`.

| Name | Type | Description | Required | Example |
|-|-|-|-|-|
| backend | str | Name of the backend to make the query. | Yes | "ibm_fez" |
| pubs | Iterable[EstimatorPubLike]  | An iterable of PUB-like (primitive unified bloc) objects, such as tuples `(circuit, observables)` or `(circuit, observables, parameter_values)`. See [Overview of PUBs](/guides/primitive-input-output#overview-of-pubs) for more information. The circuits don’t need to be ISA circuits. | Yes | (circuit, observables, parameter_values) |
| precision | float | The target precision for expectation value estimates of each run Estimator PUB that does not specify its own precision.  | No | 0.1 |
| options | dict | Input options. See `Options` section for more details. | No | `{"optimization_level": 3}` |
| instance | str | The hub/group/project to use in that format. | No | "hub1/group1/project1" |

### Options

#### Structure

Similar to Qiskit Runtime primitives, options for IBM Circuit function can be specified as a nested dictionary. Commonly used options, such as ``optimization_level`` and ``mitigation_level``, are at the first level. Other, more advanced options are grouped into different categories, such as ``resilience``.

#### Defaults

If you do not specify a value for an option, the default value specified by the service is used.

#### Mitigation level

IBM Circuit function also supports `mitigation_level`. The mitigation level specifies how much error suppression and mitigation to apply to the job. Higher levels generate more accurate results, at the expense of longer processing times. The degree of error reduction depends on the method applied. The mitigation level abstracts the detailed choice of error mitigation and suppression methods to allow users to reason about the cost/accuracy trade-off appropriate to their application. The following table shows the corresponding methods for each level.

<Admonition type="note">
While the names are similar, `mitigation_level` applies different techniques than the ones used by Estimator’s `resilience_level`.
</Admonition>

Similar to ``resilience_level`` in Qiskit Runtime Estimator, ``mitigation_level`` specifies a base set of curated options. Any options you manually specify in addition to the mitigation level are applied on top of the base set of options defined by the mitigation level. Therefore, in principle, you could set the mitigation level to 1, but then turn off measurement mitigation, although this is not advised.

| **Mitigation Level** | **Technique** |
|:-:|:-:|
| 1 [Default] | Dynamical decoupling + measurement twirling + TREX  |
| 2 | Level 1 + gate twirling + ZNE via gate folding |
| 3 | Level 1 + gate twirling + ZNE via PEA |

The following example demonstrates setting the mitigation level:

In [ ]:
options = {"mitigation_level": 2}

job = function.run(
  backend=backend.name,
  pubs=pubs,
  options=options
)

#### All available options

In addition to `mitigation_level`, the IBM Circuit function also provides a select number of advanced options that allow you to fine-tune the cost/accuracy trade-off. The following table shows all the available options:

| Option | Sub-option | Sub-sub-option | Description | Choices | Default |
|-|-|-|-|-|-|
| default_precision |  |  | The default precision to use for any PUB or `run()`<br/>call that does not specify one.<br/>Each input PUB can specify its own precision. If the `run()` method is given a precision, then that value is used for all PUBs in the `run()` call that do not specify their own.  | float > 0 | 0.015625 |
| max_execution_time |  |  | Maximum execution time in seconds, which is based<br/>on QPU usage (not wall clock time). QPU usage is the<br/>amount of time that the QPU is dedicated to processing your job. If a job exceeds this time limit, it is forcibly canceled. | Integer number of seconds in the range [1, 10800] |  |
| mitigation_level |  |  | How much error suppression and mitigation to apply. Refer to the [Mitigation level](#mitigation-level) section for more information about the methods used at each level. | 1 / 2 / 3 | 1 |
| optimization_level |  |  | How much optimization to perform on the circuits. [Higher levels](/guides/set-optimization) generate more optimized circuits, at the expense of longer transpilation time. | 0 / 1 / 2 / 3 | 2 |
| dynamical_decoupling | enable |  | Whether to enable dynamical decoupling. Refer to [Error suppression and mitigation techniques](/guides/error-mitigation-and-suppression-techniques#dynamical-decoupling) for the explanation of the method.  | True/False | True |
|  | sequence_type |  | Which dynamical decoupling sequence to use.<br/>* `XX`: use the sequence `tau/2 - (+X) - tau - (+X) - tau/2`<br/>* `XpXm`: use the sequence `tau/2 - (+X) - tau - (-X) - tau/2`<br/>* `XY4`: use the sequence<br/>`tau/2 - (+X) - tau - (+Y) - tau (-X) - tau - (-Y) - tau/2` | 'XX'/'XpXm'/'XY4' | 'XX' |
| twirling | enable_gates |  | Whether to apply 2-qubit Clifford gate twirling. | True/False | False |
|  | enable_measure |  | Whether to enable twirling of measurements. | True/False | True |
| resilience | measure_mitigation |  | Whether to enable TREX measurement error mitigation method. Refer to [Error suppression and mitigation techniques](/guides/error-mitigation-and-suppression-techniques#twirled-readout-error-extinction-trex) for the explanation of the method.  | True/False | True |
|  | zne_mitigation |  | Whether to turn on Zero Noise Extrapolation error mitigation method. Refer to [Error suppression and mitigation techniques](/guides/error-mitigation-and-suppression-techniques#zero-noise-extrapolation-zne) for the explanation of the method.  | True/False | False |
|  | zne | amplifier | Which technique to use for amplifying noise. One of: <br/> - `gate_folding` (default) uses 2-qubit gate folding to amplify noise. If the noise factor requires amplifying only a subset of the gates, then these gates are chosen randomly.<br/><br/> - `gate_folding_front` uses 2-qubit gate folding to amplify noise. If the noise factor requires amplifying only a subset of the gates, then these gates are selected from the front of the topologically ordered DAG circuit.<br/><br/> - `gate_folding_back` uses 2-qubit gate folding to amplify noise. If the noise factor requires amplifying only a subset of the gates, then these gates are selected from the back of the topologically ordered DAG circuit.<br/><br/> - `pea` uses a technique called Probabilistic error amplification (PEA) to amplify noise. Refer to [Error suppression and mitigation techniques](/guides/error-mitigation-and-suppression-techniques#probabilistic-error-amplification-pea) for the explanation of the method.  | gate_folding / gate_folding_front / gate_folding_back / pea | gate_folding |
|  |  | noise_factors | Noise factors to use for noise amplification. | list of floats; each float >= 1 | (1, 1.5, 2) for PEA, and (1, 3, 5) otherwise. |
|  |  | extrapolator | Noise factors to evaluate the fit extrapolation models at. This option does not affect execution or model fitting in any way; it only determines the points at which the `extrapolator`s are evaluated to be returned in the data fields called `evs_extrapolated` and `stds_extrapolated`. | one or more of `exponential`,`linear`, `double_exponential`,`polynomial_degree_(1 <= k <= 7)` | (`exponential`, `linear`) |
|  | pec_mitigation |  | Whether to turn on Probabilistic Error Cancellation error mitigation method. Refer to [Error suppression and mitigation techniques](/guides/error-mitigation-and-suppression-techniques#probabilistic-error-cancellation-pec) for the explanation of the method.  | True/False | False |
|  | pec | max_overhead | The maximum circuit sampling overhead allowed, or `None` for no maximum. | None/ integer >1 | 100 |

In the following example, setting the mitigation level to 1 initially turns off ZNE mitigation, but setting `zne_mitigation` to `True` overrides the relevent setup from `mitigation_level`.

In [ ]:
options = {
  "mitigation_level": 1,
  "resilience": {"zne_mitigation": True}
}

## Get support

Reach out to [IBM Quantum support](https://docs.quantum.ibm.com/support), and include the following information:

- Qiskit Function Job ID (`qiskit-ibm-catalog`), `job.job_id`
- A detailed description of the issue
- Any relevant error messages or codes
- Steps to reproduce the issue

## Next steps

<Admonition type="tip" title="Recommendations">

- Try the [Building workflows with the IBM Circuit function](https://learning.quantum.ibm.com/tutorial/building-workflows-with-the-ibm-circuit-function) tutorial.

</Admonition>